# BIA 6304 Text Mining - Assignment 1 

Author: Andrew Pierson

## T1
Create a new data frame of twitter information. Choose a search term and fetch 500 tweets. Modify your function to include only the fields you think are necessary and create a data frame (no more than five additional columns).  Don’t forget to show only the head and tail of that data frame.

### Project Background and Purpose
The purpose of this project is to gather insight on the popularity of The Kansas City Chiefs during the preseason, and to use this information to predict attendance at future games. This project will provide value to the KC Chiefs' organization by benchmarking their current popularity and allowing them the opportunity to set targets for improving their fanbase's satisfaction at games. An example of improving fan satisfaction at Arrowhead Stadium is by giving away memorabilia. In contrast, if the Chiefs organization notices that their fan's satisfaction is trending in a positive direction and is predicted to continue trending in a similar manner, they might choose to give out less free-merchandise at games. Twitter will be the first method used to determine popularity then other methods can be added to predict on a better sample from various media sites such as YouTube API, Facebook API, and Google Search API.         

In [1]:
#Import pandas for basic data frame manipulation
import pandas as pd
#Import requests to use API calls
import requests
#Import tweepy to interact with twitter API 
import tweepy
from tweepy import OAuthHandler
#Import csv to export results for visual analysis
import csv

### Tweepy OAuth Auhtentication
After registering for Twitter API development, unique key will be assigned. Below we define the keys that Twitter assigned, create an OAuthHandler instance using the consumer key and secret, then exchanging the request token for an access token allowing us to fetch information. All of this is done using the requests and tweepy packages.

In [2]:
#Define twitter developer API keys to get information from different  
consumer_key =  'tKRiH8NBRiMuexosYHjKzzfJG'
consumer_secret = 'W45BJD5QqwPu780jn8XqpweJqk3Ugd4xw8yeEr4PCZctHXC3Q9'
access_token= '185897808-IPVxscpc37tcZ5osjL4OIboAKYAjgonjssWwmbXd'
access_secret= 'GROWamgFWBBmV37H3FxXkW75WHLFSAuVo8uPaXqyBaqRT'

#Define authorization type for request
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

### Search for Tweets by Hashtag and Geospatial
In order to forecast attendance at the Kansas City Chiefs' a geospatial method will be used to search for tweets containing '#Chiefs" tag, that are within a 25 Kilometer radius of Arrowhead Stadium's Longitude and Latitude coordinates. The amount of tweets searched for in my API call is specified as 500 items and the objects are output in a list format 

In [3]:
#Define an empty dataframe to create a list of tweets that can be printed
results = []

#Define the variables to be used in the api search
latitude = 39.0427
longitude = -94.4837
max_range = 25
hashtag = '#Chiefs'

#Get the first 500 items based on the search query
for tweet in tweepy.Cursor(api.search, q = hashtag, geocode = '%f,%f,%dkm' % (latitude, longitude, max_range) ).items(500):
    results.append(tweet)

#Verify the number of list items is 500
print('API Call Output Type: ', type(results))
print('Tweet Objects: ', len(results))

API Call Output Type:  <class 'list'>
Tweet Objects:  500


### Choose Tweet Objects to mine
* Hashtags included in tweet objects will be important in this project when identifying fanbase loyalty.
* A total count of all of a user's tweets (including retweets and quote tweets) could be closely related to the number of a user's followers. In addition, users that have suspiciously small status counts might be omitted from the analysis due to misrepresentation of tweet population sample or because they are spam accounts.
* The verified tweet object is a boolean value that distinguishes people of interest, such as celebrities or politicians, from normal twitter users.
* Mentions on a tweet will help keep us informed of what people are sharing with others about The Kansas City Chiefs' Organization.
* The language on a majority of these tweets will likely be English (en), other languages have a higher chance of misrepresenting our sample data by including irrelevant tweet objects.
* I chose to include the text object from tweets that contain '#Chiefs' in order to find twitter users interested in the NFL team during the Kansas City Chiefs' preseason. This object was also chosen because it will likely provide insightful text that can be used to determine the underlying sentiment of a user towards the Chiefs. 
* The next two tweet objects that I chose to include in my data frame were the retweet and favorite counts for tweets including '#Chiefs' in their tags. Retweet and favorite counts can be useful when analyzing popularity of a tweet and will likely require different weights if a user is favoriting tweets more often than retweeting. 
* Including the created_at tweet object was done in order to allow the data frame to be sorted in chronological order upon anlysis. This might show trends in popularity or sentiment depending upon the Chiefs performance after a game. 
* A user's name and account description might also allow us to predict the tweet's sentiment, which is why I chose to mine user.name and user.description tweet objects. A case in which this may occur is if an opposing team's fans are including "#Chiefs" in a tweet that has a negative connotation. 
* The user.followers_count object was also  included and could provide interesting information, analyzing this using a forecasting method may indicate impressions on other users. 

In [4]:
#Create a function to convert a given list of tweets into a Pandas DataFrame
#The DataFrame will consist of only chosen values below

def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    DataSet['TweetHashtags'] = [tweet.entities.get('hashtags') for tweet in tweets]
    DataSet['UserStatusCount'] = [tweet.user.statuses_count for tweet in tweets]
    DataSet['UserVerified'] = [tweet.user.verified for tweet in tweets]
    #DataSet['TweetMentions'] = [tweet.entities.get('user_mentions') for tweet in tweets]
    DataSet['TweetLanguage'] = [tweet.lang for tweet in tweets]
    DataSet['TweetText'] = [tweet.text for tweet in tweets]
    DataSet['TweetRetweetCount'] = [tweet.retweet_count for tweet in tweets]
    DataSet['TweetFavoriteCount'] = [tweet.favorite_count for tweet in tweets]
    DataSet['TweetCreated'] = [tweet.created_at for tweet in tweets]
    #DataSet['userName'] = [tweet.user.name for tweet in tweets]
    #DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['UserFollowerCount'] = [tweet.user.followers_count for tweet in tweets]

    return DataSet

### Create and Populate DataFrame
Here we are going to use the predefined list of tweet objects that we chose to populate a dataframe, which is named 'tweet_frame.' The resulting dataframe's shape is 500 rows by 9 columns. Printing ten of the tweet's text object is very informative, already it is obvious that most of these tweet's are going to be a form of a retweet (RT). In addition, the user @ArrowheadPride (an account I am actually following) appears to be very popular.

In [5]:
#Pass the tweets list to create a DataFrame
tweet_frame = toDataFrame(results)
print('Dataframe Shape (Rows, Columns): ', tweet_frame.shape)

Dataframe Shape (Rows, Columns):  (500, 9)


In [6]:
tweet_frame['TweetText'][0:10]

0    Prep for the opener has officially started for...
1    RT @jeff_rosen88: Come for the INSANE Ty Hill ...
2    RT @ArrowheadPride: Patrick Mahomes on the 201...
3    RT @ArrowheadPride: Patrick Mahomes on the 201...
4    RT @ArrowheadPride: Patrick Mahomes on the 201...
5    RT @TheChiefsWire: #Chiefs vs. #Chargers Week ...
6    RT @Gezee80: 🎶Going going back to Cali Cali🎶 C...
7    RT @jeff_rosen88: Come for the INSANE Ty Hill ...
8    Come for the INSANE Ty Hill high school highli...
9    RT @ArrowheadPride: Patrick Mahomes on the 201...
Name: TweetText, dtype: object

Taking a look at the head of our tweet_frame is also very helpful, it gives a brief introduction to the various types of data that will potentially be used during analysis.  

In [7]:
tweet_frame.head()

,TweetHashtags,UserStatusCount,UserVerified,TweetLanguage,TweetText,TweetRetweetCount,TweetFavoriteCount,TweetCreated,UserFollowerCount
0,"[{'text': 'Chiefs', 'indices': [51, 58]}]",50073,False,en,Prep for the opener has officially started for...,0,1,2018-09-05 17:45:47,7384
1,[],208322,True,en,RT @jeff_rosen88: Come for the INSANE Ty Hill ...,3,0,2018-09-05 17:44:53,277126
2,"[{'text': 'Chiefs', 'indices': [57, 64]}]",19599,False,en,RT @ArrowheadPride: Patrick Mahomes on the 201...,12,0,2018-09-05 17:40:48,459
3,"[{'text': 'Chiefs', 'indices': [57, 64]}]",1206,False,en,RT @ArrowheadPride: Patrick Mahomes on the 201...,12,0,2018-09-05 17:37:58,25
4,"[{'text': 'Chiefs', 'indices': [57, 64]}]",30542,False,en,RT @ArrowheadPride: Patrick Mahomes on the 201...,12,0,2018-09-05 17:34:00,92


In [8]:
tweet_frame.tail()

,TweetHashtags,UserStatusCount,UserVerified,TweetLanguage,TweetText,TweetRetweetCount,TweetFavoriteCount,TweetCreated,UserFollowerCount
495,"[{'text': 'Chiefs', 'indices': [26, 33]}]",26478,True,en,Here's the 10 players the #Chiefs sign to the ...,1,2,2018-09-02 20:37:51,8913
496,"[{'text': 'Chiefs', 'indices': [20, 27]}]",7706,False,en,RT @ArrowheadPride: #Chiefs make flurry of mov...,44,0,2018-09-02 20:37:43,1176
497,[],186730,False,en,RT @LWorthySports: S Ron Parker has officially...,10,0,2018-09-02 20:37:41,1124
498,"[{'text': 'Chiefs', 'indices': [20, 27]}]",139,False,en,RT @ArrowheadPride: #Chiefs make flurry of mov...,44,0,2018-09-02 20:37:16,17
499,"[{'text': 'Chiefs', 'indices': [20, 27]}]",5124,False,en,RT @ArrowheadPride: #Chiefs make flurry of mov...,44,0,2018-09-02 20:37:10,980


## T2
Using the data from Task 1, instantiate a count vectorizer to create a bag of words. Use the default to convert to lower case.  Generate a vector space model. Try various combinations of vectorizer settings like we did in class: changing case, eliminating stop words, using min and max document frequency settings, choosing n-grams, applying Tfidf-weights, etc. Try at least 4 different versions. 

In [9]:
#Import numpy into the workspace
import numpy as np
#Import the CountVectorizer module
from sklearn.feature_extraction.text import CountVectorizer
#Import the math function to do math
import math
#Specify the max width of pixels per column for text that will be analyzed
pd.set_option('display.max_colwidth', 800)

### Bag of Words
The first step to instantiate a count vectorizer to create a bag of words is to define the type of vectorizer method. In this case we want to retrieve the words from the tweet text by defining the binary vectorizer setting as false. By default the CountVectorizer will also convert all of the text to lowercase. After fitting the count vectorizer to the tweet text feature, the shape of the resulting frame has a 500 by 924 sparse matrix feature space.

#### Feature Space Model
cv1: Default Parameters

In [10]:
#Define the count vectorizer using default parameters
cv1 = CountVectorizer()
#Apply the count vectorizer for the bag of words to the dataframe feature
cv1_text = cv1.fit_transform(tweet_frame['TweetText'])
#This is the feature space for TweetText
print('Shape:', cv1_text.shape)
#This is the type of matrix that is returned for TweetText
print('Type:', type(cv1_text))

Shape: (500, 1009)
Type: <class 'scipy.sparse.csr.csr_matrix'>


In [11]:
#Print the sparse matrix as a data frame for an aestheticallly pleasing view of the feature space head
pd.DataFrame(cv1_text.toarray(), columns = cv1.get_feature_names()).head(15)

,0c1fxbly0n,0lpdcqfxjq,10,11,12,120,14,16,18,19,...,xmj2ig8a62,xwkhqyy3qi,xza9cwrhv9,yea,year,years,you,your,zimmerman,zrku42c0q6
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#The "bag of words" from the TweetText feature space displayed as a list
print(cv1.get_feature_names())

['0c1fxbly0n', '0lpdcqfxjq', '10', '11', '12', '120', '14', '16', '18', '19', '1lgrqrnrlc', '1qphvva45r', '2018', '21', '22glu73ehw', '27', '30', '32qp1aadux', '3rd', '41actionnews', '4haqobifew', '4iejthsjvd', '4th', '4ydbhdtmma', '53', '5agdtxn1pc', '5lezvyefkm', '60', '610sportskc', '68c4gyto5d', '6fkgjgonfe', '6pm', '77', '7faxy2q988', '7j8vj9wmap', '7th', '7uinfq3uxu', '7ujctigvfk', '810borderpatrol', '810wh', '98jzpdxh42', '9bueur0d97', '9cwmd7iq6b', '9vhb6essw0', 'a12msaqvut', 'abcs', 'about', 'acquiring', 'active', 'adamteicher', 'add', 'added', 'adjusted', 'after', 'afternoon', 'again', 'against', 'agent', 'ago', 'alex', 'alexander', 'all', 'allow', 'almost', 'along', 'am', 'amerson', 'amp', 'an', 'and', 'andrew', 'andy', 'angeles', 'announce', 'another', 'anthony', 'any', 'are', 'arm', 'around', 'arrow', 'arrowhead', 'arrowheadlines', 'arrowheadphones', 'arrowheadpride', 'arrowheadreport', 'as', 'asmcda637n', 'at', 'athletic', 'austin', 'away', 'awesome', 'b5gilse31u', 'back'

#### Vector Space Model
cv2: binary = 'False', lowercase = 'True'

In [13]:
#Define the count vectorizer by setting the binary parameter to 'False' and the lowercase as its default, 'True'
cv2 = CountVectorizer(binary=False, lowercase = 'True')
#Apply the count vectorizer for the bag of words to the dataframe feature
cv2_text = cv2.fit_transform(tweet_frame['TweetText'])
#This is the vector space for TweetText
print('Shape:', cv2_text.shape)
#This is the type of matrix that is returned for TweetText
print('Type:', type(cv2_text))

Shape: (500, 1009)
Type: <class 'scipy.sparse.csr.csr_matrix'>


In [14]:
#Print the sparse matrix as a data frame for an aestheticallly pleasing view of the vector space head 
pd.DataFrame(cv2_text.toarray(),columns = cv2.get_feature_names()).head(15)

,0c1fxbly0n,0lpdcqfxjq,10,11,12,120,14,16,18,19,...,xmj2ig8a62,xwkhqyy3qi,xza9cwrhv9,yea,year,years,you,your,zimmerman,zrku42c0q6
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#The "bag of words" from the TweetText vector space displayed as a list
print(cv2.get_feature_names())

['0c1fxbly0n', '0lpdcqfxjq', '10', '11', '12', '120', '14', '16', '18', '19', '1lgrqrnrlc', '1qphvva45r', '2018', '21', '22glu73ehw', '27', '30', '32qp1aadux', '3rd', '41actionnews', '4haqobifew', '4iejthsjvd', '4th', '4ydbhdtmma', '53', '5agdtxn1pc', '5lezvyefkm', '60', '610sportskc', '68c4gyto5d', '6fkgjgonfe', '6pm', '77', '7faxy2q988', '7j8vj9wmap', '7th', '7uinfq3uxu', '7ujctigvfk', '810borderpatrol', '810wh', '98jzpdxh42', '9bueur0d97', '9cwmd7iq6b', '9vhb6essw0', 'a12msaqvut', 'abcs', 'about', 'acquiring', 'active', 'adamteicher', 'add', 'added', 'adjusted', 'after', 'afternoon', 'again', 'against', 'agent', 'ago', 'alex', 'alexander', 'all', 'allow', 'almost', 'along', 'am', 'amerson', 'amp', 'an', 'and', 'andrew', 'andy', 'angeles', 'announce', 'another', 'anthony', 'any', 'are', 'arm', 'around', 'arrow', 'arrowhead', 'arrowheadlines', 'arrowheadphones', 'arrowheadpride', 'arrowheadreport', 'as', 'asmcda637n', 'at', 'athletic', 'austin', 'away', 'awesome', 'b5gilse31u', 'back'

#### Count Vectorizer 3
cv3: binary = 'False', stop_words = 'english', lowercase = 'True'

In [16]:
#Define the count vectorizer by setting the binary parameter as 'false', stop_words as 'english', and lowercase as 'true'
cv3 = CountVectorizer(binary=False, stop_words = 'english', lowercase = 'True')
#Apply the count vectorizer for the bag of words to the dataframe feature
cv3_text = cv3.fit_transform(tweet_frame['TweetText'])
#This is the vector space for TweetText
print('Type:', type(cv3_text))
#This is the type of matrix that is returned for TweetText
print('Shape:', cv3_text.shape)
#Print the sparse matrix as a data frame for an aestheticallly pleasing view of the vector space head 
pd.DataFrame(cv3_text.toarray(), columns = cv3.get_feature_names()).head(15)

Type: <class 'scipy.sparse.csr.csr_matrix'>
Shape: (500, 872)


,0c1fxbly0n,0lpdcqfxjq,10,11,12,120,14,16,18,19,...,wylie,xkg4aoimfj,xmj2ig8a62,xwkhqyy3qi,xza9cwrhv9,yea,year,years,zimmerman,zrku42c0q6
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Count Vectorizer 4
cv4: binary = 'False', stop_words = 'english', lowercase = 'False'

In [17]:
#Define the count vectorizer by setting the binary parameter as 'false', stop_words as 'english', and lowercase as 'true'
cv4 = CountVectorizer(binary=False, stop_words = 'english', lowercase = 'False')
#Apply the count vectorizer for the bag of words to the dataframe feature
cv4_text = cv4.fit_transform(tweet_frame['TweetText'])
#This is the vector space for TweetText
print('Type:', type(cv4_text))
#This is the type of matrix that is returned for TweetText
print('Shape:', cv4_text.shape)
#Print the sparse matrix as a data frame for an aestheticallly pleasing view of the vector space head 
pd.DataFrame(cv4_text.toarray(), columns = cv4.get_feature_names()).head(15)

Type: <class 'scipy.sparse.csr.csr_matrix'>
Shape: (500, 872)


,0c1fxbly0n,0lpdcqfxjq,10,11,12,120,14,16,18,19,...,wylie,xkg4aoimfj,xmj2ig8a62,xwkhqyy3qi,xza9cwrhv9,yea,year,years,zimmerman,zrku42c0q6
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Count Vectorizer 5
cv5: binary = 'False', max_df = 0.85 

In [18]:
#Define the count vectorizer by setting the binary parameter as 'false', stop_words as 'english', and max_df = 0.85
cv5 = CountVectorizer(binary=False, stop_words = 'english', max_df = 0.85)
#Apply the count vectorizer for the bag of words to the dataframe feature
cv5_text = cv5.fit_transform(tweet_frame['TweetText'])
#This is the vector space for TweetText
print('Type:', type(cv5_text))
#This is the type of matrix that is returned for TweetText
print('Shape:', cv5_text.shape)
#Print the sparse matrix as a data frame for an aestheticallly pleasing view of the vector space head 
pd.DataFrame(cv5_text.toarray(), columns = cv5.get_feature_names()).head(15)

Type: <class 'scipy.sparse.csr.csr_matrix'>
Shape: (500, 871)


,0c1fxbly0n,0lpdcqfxjq,10,11,12,120,14,16,18,19,...,wylie,xkg4aoimfj,xmj2ig8a62,xwkhqyy3qi,xza9cwrhv9,yea,year,years,zimmerman,zrku42c0q6
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Count Vectorizer 6
cv6: binary = 'False', min_df = 0.10, stop_words = 'english'

In [19]:
#Define the count vectorizer by setting the binary parameter as 'false', stop_words as 'english', and min_df = 0.10
cv6 = CountVectorizer(binary=False, stop_words = 'english', min_df = 0.10)
#Apply the count vectorizer for the bag of words to the dataframe feature
cv6_text = cv6.fit_transform(tweet_frame['TweetText'])
#This is the vector space for TweetText
print('Type:', type(cv6_text))
#This is the type of matrix that is returned for TweetText
print('Shape:', cv6_text.shape)
#Print the sparse matrix as a data frame for an aestheticallly pleasing view of the vector space head 
pd.DataFrame(cv6_text.toarray(), columns = cv6.get_feature_names()).head(15)

Type: <class 'scipy.sparse.csr.csr_matrix'>
Shape: (500, 19)


,arrowheadpride,bowl,chart,chiefs,depth,e4pkzm278p,https,illustrated,litton,ol,practice,predict,qb,rt,sports,squad,super,win,writers
0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0


### Count Vectorizer Summary
* cv1: Default Parameters
* cv2: binary = 'False', lowercase = 'True'
* cv3: binary = 'False', stop_words = 'english', lowercase = 'True'
* cv4: binary = 'False', stop_words = 'english', lowercase = 'False'
* cv5: binary = 'False', max_df = 0.85  
* cv6: binary = 'False', min_df = 0.10, stop_words = 'english'